In [2]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras import Sequential
from keras.layers import Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
import keras

In [3]:
model = keras.models.load_model('../input/zalo-2021-task-1-dataset/public_test/masknet.h5')

In [4]:
mask_label = {0:'MASK',1:'NO MASK'}
dist_label = {0:(0,255,0),1:(255,0,0)}

In [5]:
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance

In [6]:
def check(index):
    fname = f'../input/zalo-2021-task-1-dataset/public_test/images/{index}.jpg'
    fface = f'../input/zalo-2021-task-1-dataset/public_test/NEW_INPUT/NEW_INPUT/{index}.json'
    
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.IMREAD_GRAYSCALE)
    
    faces = []
    
    with open(fface, 'r') as f:
        info_faces = json.load(f)
        
        boxes = info_faces['boxes']
        score = info_faces['score']
        
        for i in range(len(boxes)):
            if (score[i] > 0.94):
                faces.append([int(v) for v in boxes[i]])
    
    
    #label = [0 for i in range(len(faces))]
    #for i in range(len(faces)-1):
    #    for j in range(i+1, len(faces)):
    #        dist = distance.euclidean(faces[i][:2],faces[j][:2])
    #        if dist<MIN_DISTANCE:
    #            label[i] = 1
    #            label[j] = 1

    new_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    result = 1

    for i in range(len(faces)):
        (x1, y1, x2, y2) = faces[i]
        #crop = new_img[y:y+h,x:x+w]
        #crop = cv2.resize(crop,(128,128))
        #crop = np.reshape(crop,[1,128,128,3])/255.0
        #mask_result = model.predict(crop)
        #cv2.putText(new_img,mask_label[mask_result.argmax()],(x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,dist_label[label[i]],2)
        cv2.rectangle(new_img,(y1,x1),(y2,x2), (0,255,0), 1)#,dist_label[label[i]],1)
    plt.figure(figsize=(10,10))
    plt.imshow(new_img)

In [7]:
MIN_DISTANCE = 1300

In [8]:
check(59)

In [13]:
with open('output.csv', 'w') as f:
    f.write('image_id,fname,5K\n')
for index in range(1, 543):
    fname = f'../input/zalo-2021-task-1-dataset/public_test/images/{index}.jpg'
    fface = f'../input/zalo-2021-task-1-dataset/public_test/NEW_INPUT/NEW_INPUT/{index}.json'
    
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.IMREAD_GRAYSCALE)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    faces = []
    
    with open(fface, 'r') as f:
        info_faces = json.load(f)
        
        boxes = info_faces['boxes']
        score = info_faces['score']
        
        for i in range(len(boxes)):
            if (score[i] > 0.94):
                x1, y1, x2, y2 = boxes[i]
                
                x1 = int(max(x1, 0)); x2 = int(min(x2, img.shape[1] - 1))
                y1 = int(max(y1, 0)); y2 = int(min(y2, img.shape[0] - 1))
                
                if (x1 < x2 and y1 < y2):
                    faces.append((x1, y1, x2, y2))
        
    Mask_label = 1
    Dist_label = 1
    
    for (x1, y1, x2, y2) in faces:
        crop = img[y1:y2, x1:x2]
        crop = cv2.resize(crop,(128,128))
        crop = np.reshape(crop,[1,128,128,3])/255.0
        mask_result = model.predict(crop)
        
        if (mask_result.argmax() == 1):
            Mask_label = 0
            break
            
    for i in range(len(faces)):
        for j in range(i):
            cen_xi = (faces[i][0] + faces[i][2]) / 2; cen_yi = (faces[i][1] + faces[i][3]) / 2
            cen_xj = (faces[j][0] + faces[j][2]) / 2; cen_yj = (faces[j][1] + faces[j][3]) / 2
            
            #cen_xi = 100.0 * cen_xi / (faces[i][2] - faces[i][0]); cen_yi = 100.0 * cen_yi / (faces[i][2] - faces[i][0])
            #cen_xj = 100.0 * cen_xj / (faces[j][2] - faces[j][0]); cen_yj = 100.0 * cen_yj / (faces[j][2] - faces[j][0])
            
            if  distance.euclidean([cen_xi, cen_yi], [cen_xj, cen_yj]) < 130:
                Dist_label = 0
                break
        
        if  Dist_label == 0:
            break
    
    with open('output.csv', 'a') as f:
        f.write(f'{index},{index}.jpg,{Mask_label & Dist_label}\n')